# Text generation using RNN - Word Level

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously. He was not aware of the situation at all."

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X                |  Y      |
|-----------------------|---------|
|    < word1 >< word2 > | < word3 > |
|    Her brother        |  shook  |
|    brother shook      |  his    |
|    shook his          |  head   |
|    his head           | incredulously |
|    head incredulously |    .    |
|    ..                 |    .    |
|    situation at       |  all    |
|    at all             |    .    |

Note that in the above problem, the sequence length of **X is two words** and that of **y is one word**. Hence, this is a many-to-one architecture. We can, however, change the number of input words to any number depending on the problem.

A model is trained on such data. To generate text, we simply give the model any two words using which it predicts the next word. Then it appends the predicted word to the input sequence (to the extreme right of the sequence) and discards the first word (word on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "Did I"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        Did I                      |    < predicted word 1 >  |
|               I < predicted word 1 >              |    < predicted word 2 >  |
|       < predicted word 1 > < predicted word 2 >   |    < predicted word 3 >  |
|       < predicted word 2 > < predicted word 3 >   |    < predicted word 4 >  |
|                      ...                          |            ...           | 

# Notebook Overview
1. Preprocess data
2. Build LSTM model
3. Generate text

In [1]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import requests
from nltk.tokenize import word_tokenize

from gensim.models import KeyedVectors

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


# 1. Preprocess data

In [2]:
# download ebook
url = "https://www.gutenberg.org/files/24869/24869-0.txt"
book = requests.get(url)
data = book.text

In [3]:
# let's look at the text
print(data[:500])

ï»¿The Project Gutenberg EBook of The Ramayana



This eBook is for the use of anyone anywhere at no cost and with almost no
restrictions whatsoever. You may copy it, give it away or re-use it under
the terms of the Project Gutenberg License included with this eBook or
online at http://www.gutenberg.org/license



Title: The Ramayana



Release Date: March 18, 2008 [Ebook #24869]

Language: English

Character set encoding: UTF-8


***START OF THE PROJECT GUTENBERG EBOOK THE


In [4]:
# subset the book from the first chapter, that ism INVOCATION - everything before first chapter is irrelevant data
start_index = re.search("invocation.\(1\)", data, re.I)
print(start_index.start())

19177


In [5]:
# Let's see how does the text look like
data = data[start_index.start():]

In [6]:
# let's look at the text
print(data[:500])

INVOCATION.(1)


Praise to VÃ¡lmÃ­ki,(2)bird of charming song,(3)
  Who mounts on Poesyâs sublimest spray,
And sweetly sings with accent clear and strong
  RÃ¡ma, aye RÃ¡ma, in his deathless lay.

Where breathes the man can listen to the strain
  That flows in music from VÃ¡lmÃ­kiâs tongue,
Nor feel his feet the path of bliss attain
  When RÃ¡maâs glory by the saint is sung!

The stream RÃ¡mÃ¡yan leaves its sacred fount
  The whole wide world from sin and stain to free.(4)
T


## Clean text

In [7]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91975\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
import regex

In [9]:
# define a function to clean text data
def clean_document(document, char_filter = r"[^\w]"):
    '''
    input:
    document          :  string
    char_filter       :  regex pattern - removes those characters from the text that match the pattern

    output: clean document
    '''
    
    # convert words to lower case
    document = document.lower()

    # tokenise words
    words = word_tokenize(document)

    # strip whitespace from all words
    words = [word.strip() for word in words]

    # join back words to get document
    document = " ".join(words)

    # remove unwanted characters
    document = regex.sub(char_filter, " ", document)

    # replace multiple whitespaces with single whitespace
    document = regex.sub(r"\s+", " ", document)

    # strip whitespace from document
    document = document.strip()

    return document

data = clean_document(data)

In [10]:
# length of text
words = word_tokenize(data)
print("Number of words in document: {}".format(len(words)))

Number of words in document: 427304


## Convert characters to integers

In [11]:
# use Keras' Tokenizer() function to encode text to integers
word_tokeniser = Tokenizer()
word_tokeniser.fit_on_texts([data])
encoded_words = word_tokeniser.texts_to_sequences([data])[0]

In [12]:
# check the size of the vocabulary
VOCABULARY_SIZE = len(word_tokeniser.word_index) + 1
print('Vocabulary Size: {}'.format(VOCABULARY_SIZE))

Vocabulary Size: 18415


## Divide data in X and y

### Create sequences

In each training sample, X will have a sequence of 5 words and y will have the sixth word. In other words, this means that use previous five words of a sequence to predict next word.

In [13]:
sequences = []
MAX_SEQ_LENGTH = 5  # X will have five words, y will have the sixth word

for i in range(MAX_SEQ_LENGTH, len(encoded_words)):
    sequence = encoded_words[i-MAX_SEQ_LENGTH:i+1]
    sequences.append(sequence)
sequences = np.array(sequences)

In [14]:
print('Total number of training samples: {}'.format(len(sequences)))
print('\nSample sequences: \n{}'.format(sequences[0:3]))

Total number of training samples: 427133

Sample sequences: 
[[7128  842  887    4   54 1095]
 [ 842  887    4   54 1095 1353]
 [ 887    4   54 1095 1353 1999]]


In [15]:
# divide the sequence into X and y
sequences = np.array(sequences)

X = sequences[:,:-1]  # assign all but last words of a sequence to X
y = sequences[:,-1]   # assign last word of each sequence to y

In [16]:
# Look at the first training example
print("Input of the first data point:", X[0], "\n")
print("Output of the first data point: [", y[0], "]")

Input of the first data point: [7128  842  887    4   54] 

Output of the first data point: [ 1095 ]


### One-hot encode y

In [17]:
y.shape

(427133,)

In [18]:
y = to_categorical(y, num_classes=VOCABULARY_SIZE)

MemoryError: Unable to allocate 29.3 GiB for an array with shape (427133, 18415) and data type float32

In [ ]:
print(X.shape)
print(y.shape)

There are 410241 sequences (data points) in total.

Remember that to use an RNN data has to be of the shape (#samples, #timesteps, #features)

In X, the third dimension, that is, number of features is missing because we're going to use the Keras' Embedding Layer. Hence we don't need to explicitly reshape the data to incorporate the third dimension. That will be done automatically by Keras.

In y, the second dimension is missing, that is, the number of timesteps because y is not a sequence, it's just a single word. The number of features are represented by a one-hot encoded vector whose length is the VOCABULARY_SIZE.

### Pad sequences

In [ ]:
X = pad_sequences(X, maxlen=MAX_SEQ_LENGTH, padding='pre')
print('Input sequence length: {}'.format(MAX_SEQ_LENGTH))

# 2. LSTM

In [ ]:
# create model architecture

EMBEDDING_SIZE = 100


model = Sequential()

# embedding layer
model.add(Embedding(VOCABULARY_SIZE, EMBEDDING_SIZE, input_length = MAX_SEQ_LENGTH))

# lstm layer 1
model.add(LSTM(128, return_sequences=True))

# lstm layer 2
model.add(LSTM(128))

# output layer
model.add(Dense(VOCABULARY_SIZE, activation='softmax'))

In [ ]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
# summarize defined model
model.summary()

In [ ]:
# fit network
model.fit(X, y, epochs=10, verbose=2, batch_size=256)

### Load word embeddings to represent the input words

In [ ]:
# word2vec download link (Size ~ 1.5GB): https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

path = 'storage/word-embeddings/GoogleNews-vectors-negative300.bin'

# load word2vec using the following function present in the gensim library
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
# assign word vectors from word2vec model

EMBEDDING_SIZE  = 300  # each word in word2vec model is represented using a 300 dimensional vector
VOCABULARY_SIZE = len(word_tokeniser.word_index) + 1

# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))

# create a word to index dictionary mapping
word2id = word_tokeniser.word_index

# copy vectors from word2vec model to the words present in corpus
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass

In [ ]:
# check embedding dimension
print("Embeddings shape: {}".format(embedding_weights.shape))

In [ ]:
# create model architecture

model_wv = Sequential()

# embedding layer
model_wv.add(Embedding(VOCABULARY_SIZE, EMBEDDING_SIZE, input_length = MAX_SEQ_LENGTH,
                    weights = [embedding_weights], trainable=True))

# lstm layer 1
model_wv.add(LSTM(128, return_sequences=True))

# lstm layer 2
# when using multiple LSTM layers, set return_sequences to True at the previous layer
# because the current layer expects a sequential intput rather than a single input
model_wv.add(LSTM(128))

# output layer
model_wv.add(Dense(VOCABULARY_SIZE, activation='softmax'))

In [ ]:
# compile network
model_wv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
# summarize defined model
model_wv.summary()

In [ ]:
# fit network
model_wv.fit(X, y, epochs=10, batch_size=256)

# 3. Generate text

In [ ]:
# generate a sequence from a language model
def generate_words(model, word_tokeniser, MAX_SEQ_LENGTH, seed, n_words):
    
    text = seed
    
    # generate n_words
    for _ in range(n_words):
        
        # encode text as integers
        encoded_words = word_tokeniser.texts_to_sequences([text])[0]
        
        # pad sequences
        padded_words = pad_sequences([encoded_words], maxlen=MAX_SEQ_LENGTH, padding='pre')
        
        # predict next word
        prediction = model.predict_classes(padded_words, verbose=0)
        
        # convert predicted index to its word
        next_word = ""
        for word, i in word_tokeniser.word_index.items():
            if i == prediction:
                next_word = word
                break
        
        # append predicted word to text
        text += " " + next_word
        
    return text

### Let's look at some text generations

In [ ]:
# text generation using first model - model without word embeddings
seed_text = "rama never told anyone about"
num_words = 100
print(generate_words(model, word_tokeniser, MAX_SEQ_LENGTH, seed_text, num_words))

In [ ]:
# text generation using second model - model with word embeddings
seed_text = "rama never told anyone about"
num_words = 100
print(generate_words(model_wv, word_tokeniser, MAX_SEQ_LENGTH, seed_text, num_words))

In [ ]:
# text generation using first model - model without word embeddings
seed_text = "how are you doing"
num_words = 100
print(generate_words(model, word_tokeniser, MAX_SEQ_LENGTH, seed_text, num_words))

In [ ]:
# text generation using second model - model with word embeddings
seed_text = "how are you doing"
num_words = 100
print(generate_words(model_wv, word_tokeniser, MAX_SEQ_LENGTH, seed_text, num_words))